In [33]:
import pandas as pd
import numpy as np
import geopandas

In [34]:
iris_talence_latlon = geopandas.GeoDataFrame.from_file('original_data/iris/iris_talence.js')
iris_talence_latlon.head()

,DCOMIRIS,DEPCOM,IRIS,NOM_COM,NOM_IRIS,TYP_IRIS,geometry
0,335220112,33522,0112,Talence,Thouars 2,H,POLYGON ((-0.587543196053298 44.78908756888363...
1,335220102,33522,0102,Talence,La Fauvette,H,POLYGON ((-0.587032337621043 44.81515751477936...
2,335220108,33522,0108,Talence,Peylanne-Leysotte,H,POLYGON ((-0.5828804443461369 44.8048544065047...
3,335220103,33522,0103,Talence,Combattants-Bijou,H,POLYGON ((-0.590734733544139 44.78731246999818...
4,335220105,33522,0105,Talence,Cauderes,H,POLYGON ((-0.587032337621043 44.81515751477936...


In [35]:
activite_resident = pd.read_excel('original_data/iris/activite-residents-2012.xls', skiprows=5)
activite_resident.head()

,IRIS,REG,REG2016,DEP,UU2010,COM,LIBCOM,TRIRIS,GRD_QUART,LIBIRIS,...,P12_ACTOCC15P_ILT3,P12_ACTOCC15P_ILT4,P12_ACTOCC15P_ILT5,P12_ACTOCC15P_ILT45D,C12_ACTOCC15P,C12_ACTOCC15P_PAS,C12_ACTOCC15P_MAR,C12_ACTOCC15P_DROU,C12_ACTOCC15P_VOIT,C12_ACTOCC15P_TCOM
0,010010000,82,84,01,01000,01001,L'Abergement-Clémenciat,ZZZZZZ,0100100,L'Abergement-Clémenciat (commune non irisée),...,83.676923,14.942308,0.000000,NaN,338.692308,31.876923,7.969231,3.984615,290.876923,3.984615
1,010020000,82,84,01,01000,01002,L'Abergement-de-Varey,ZZZZZZ,0100200,L'Abergement-de-Varey (commune non irisée),...,28.119658,2.008547,0.000000,NaN,104.444444,0.000000,0.000000,0.000000,96.410256,8.034188
2,010040101,82,84,01,01302,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Les Perouses-Triangle d'Activite,...,112.198631,4.828213,0.000000,NaN,715.635129,75.289455,119.660483,21.756694,407.353725,91.574773
3,010040102,82,84,01,01302,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Longeray-Gare,...,251.078104,1.056829,0.000000,NaN,1478.540370,57.685313,168.621858,26.249674,1003.722497,222.261029
4,010040201,82,84,01,01302,01004,Ambérieu-en-Bugey,ZZZZZZ,0100402,Centre-St Germain-Vareilles,...,303.454850,12.282740,1.049194,NaN,1528.972551,26.832875,139.834053,47.053809,1123.438076,191.813737


In [36]:
import unicodedata

def unicode_to_string(word):
    if type(word) == str:
        return word
    elif pd.isnull(word):
        return 
    else:
        return unicodedata.normalize('NFKD', word).encode('ascii','ignore')
    
def data_to_string(data):
    new_data = data.copy()
    
    col_title_unicode = new_data.columns.values.tolist()
    col_title = map(unicode_to_string,col_title_unicode) 
    new_data.columns = col_title
    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == unicode: #check the first not-NaN value
            new_data[col] = map(unicode_to_string,new_data[col])
            
    return new_data

In [37]:
def rename_column(data): 
    new_data = data.copy()
    col_title = new_data.columns.tolist()
    for x in xrange(len(col_title)):
        if col_title[x] == 'IRIS':
            col_title[x] = 'DCOMIRIS' # this will be the pivot column for merging
        if col_title[x] == 'LIBIRIS':
            col_title[x] = 'NOM_IRIS'
        if col_title[x] == "COM":
            col_title[x] = "DEPCOM"
        if col_title[x] == "LIBCOM": 
            col_title[x] = "NOM_COM"
    new_data.columns = col_title
    return new_data

In [38]:
def to_digit(string):
    new_string = list(string)[:]
    for x in xrange(len(new_string)): 
        if not new_string[x].isdigit():
            new_string[x] = '0'
    return int(''.join(new_string))

def convert_code(data): # convert code from str to int and replace 2AXXX or 2BXXX by 20XXX 
        new_data = data.copy()

        iris_list = new_data['DCOMIRIS'].tolist() 
        new_iris_list = [int(code) if code.isdigit() else to_digit(code) for code in iris_list]
        new_data['DCOMIRIS'] = new_iris_list
        
        depcom_list = new_data['DEPCOM'].tolist() 
        new_depcom_list = [int(code) if code.isdigit() else to_digit(code) for code in depcom_list]
        new_data['DEPCOM'] = new_depcom_list

        return new_data

In [39]:
def float_to_int(data):
    new_data = data.copy()    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == np.float64: #check the first not-NaN value
            new_data[col] = new_data[col].round()
    return new_data

In [40]:
iris_talence_latlon = data_to_string(iris_talence_latlon)
iris_talence_latlon = convert_code(iris_talence_latlon)

In [41]:
activite_resident = data_to_string(activite_resident)
activite_resident = rename_column(activite_resident)

In [42]:
activite_resident = convert_code(activite_resident)
activite_resident = float_to_int(activite_resident)

In [43]:
def extract_df(data, variable_list, variable_name):
    new_data = data.copy()
    new_data = new_data.loc[:,variable_list]
    
    new_data.columns = variable_name
    return new_data

In [44]:
variable_list = ['DCOMIRIS','P12_POP1564','C12_ACT1564','C12_ACT1564_CS1','C12_ACT1564_CS2',
                 'C12_ACT1564_CS3','C12_ACT1564_CS4','C12_ACT1564_CS5','C12_ACT1564_CS6','P12_CHOM1564','P12_RETR1564']
variable_name = ['DCOMIRIS', 'Population', 'Actifs','Agriculteurs','Artisans','Cadres',
                'Prof_intermediaires','Employes', 'Ouvriers','Chomeurs','Retraites']
act_resident = extract_df(activite_resident, variable_list, variable_name)

In [45]:
act_resident[act_resident.DCOMIRIS == 335220112]

,DCOMIRIS,Population,Actifs,Agriculteurs,Artisans,Cadres,Prof_intermediaires,Employes,Ouvriers,Chomeurs,Retraites
16695,335220112,1276,883,0,21,161,237,253,200,214,76


In [46]:
talence_iris_data = pd.merge(iris_talence_latlon, act_resident, on="DCOMIRIS", how="inner")

In [47]:
talence_iris_data.head()

,DCOMIRIS,DEPCOM,IRIS,NOM_COM,NOM_IRIS,TYP_IRIS,geometry,Population,Actifs,Agriculteurs,Artisans,Cadres,Prof_intermediaires,Employes,Ouvriers,Chomeurs,Retraites
0,335220112,33522,0112,Talence,Thouars 2,H,POLYGON ((-0.587543196053298 44.78908756888363...,1276,883,0,21,161,237,253,200,214,76
1,335220102,33522,0102,Talence,La Fauvette,H,POLYGON ((-0.587032337621043 44.81515751477936...,2824,1860,0,69,498,625,471,177,219,126
2,335220108,33522,0108,Talence,Peylanne-Leysotte,H,POLYGON ((-0.5828804443461369 44.8048544065047...,2635,1837,0,77,501,584,506,135,200,205
3,335220103,33522,0103,Talence,Combattants-Bijou,H,POLYGON ((-0.590734733544139 44.78731246999818...,2655,871,5,24,171,229,276,106,243,45
4,335220105,33522,0105,Talence,Cauderes,H,POLYGON ((-0.587032337621043 44.81515751477936...,2258,1578,5,79,501,544,297,148,125,147


In [48]:
def calculate_percentage(data, population, variable_list): 
    new_data = data.copy()
    for variable in variable_list:
        new_data[variable] = 100.0*new_data[variable]/data[population]
    return new_data

In [49]:
variable_list = ['Agriculteurs','Artisans','Cadres','Prof_intermediaires','Employes','Ouvriers','Chomeurs','Retraites']
population_variable = 'Population'
talence_iris_data = calculate_percentage(talence_iris_data, population_variable, variable_list)

In [60]:
talence_iris_data.drop('Actifs',axis=1,inplace=True)

In [61]:
talence_iris_data.head()

,DCOMIRIS,DEPCOM,IRIS,NOM_COM,NOM_IRIS,TYP_IRIS,geometry,Population,Agriculteurs,Artisans,Cadres,Prof_intermediaires,Employes,Ouvriers,Chomeurs,Retraites
0,335220112,33522,0112,Talence,Thouars 2,H,POLYGON ((-0.587543196053298 44.78908756888363...,1276,0.000000,1.645768,12.617555,18.573668,19.827586,15.673981,16.771160,5.956113
1,335220102,33522,0102,Talence,La Fauvette,H,POLYGON ((-0.587032337621043 44.81515751477936...,2824,0.000000,2.443343,17.634561,22.131728,16.678470,6.267705,7.754958,4.461756
2,335220108,33522,0108,Talence,Peylanne-Leysotte,H,POLYGON ((-0.5828804443461369 44.8048544065047...,2635,0.000000,2.922201,19.013283,22.163188,19.203036,5.123340,7.590133,7.779886
3,335220103,33522,0103,Talence,Combattants-Bijou,H,POLYGON ((-0.590734733544139 44.78731246999818...,2655,0.188324,0.903955,6.440678,8.625235,10.395480,3.992467,9.152542,1.694915
4,335220105,33522,0105,Talence,Cauderes,H,POLYGON ((-0.587032337621043 44.81515751477936...,2258,0.221435,3.498671,22.187777,24.092117,13.153233,6.554473,5.535872,6.510186


In [62]:
talence_iris_data = geopandas.GeoDataFrame(talence_iris_data)
talence_iris_data.to_file('result/talence_iris_data', driver="ESRI Shapefile")